In [ ]:
import os 
import cv2
import numpy as np
import neuroglancer
from skimage import io, measure
import matplotlib.pyplot as plt

viewer = neuroglancer.Viewer()
print(viewer)

In [ ]:
def mask_to_shell(mask):
    sub_contours = measure.find_contours(mask, 1)

    sub_shells = []
    for sub_contour in sub_contours:
        sub_contour.T[[0, 1]] = sub_contour.T[[1, 0]]
        pts = sub_contour.astype(np.int32).reshape((-1, 1, 2))

        sub_shell = np.zeros(mask.shape, dtype='uint8')
        sub_shell = cv2.polylines(sub_shell, [pts], True, 1, 5, lineType=cv2.LINE_AA)
        sub_shells.append(sub_shell)
    shell = np.array(sub_shells).sum(axis=0)
    
    return shell

In [ ]:
VOL_DIR = '/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/DK52/preps/CH1/thumbnail_aligned'
files = sorted(os.listdir(VOL_DIR))

volume = []
for file in files:
    tif = io.imread(os.path.join(VOL_DIR, file))
    tif = mask_to_shell(tif)
    volume.append(tif)
volume = np.array(volume).astype('uint8')
volume = np.swapaxes(volume, 0, 2)

In [ ]:
layer = neuroglancer.SegmentationLayer(
    source = neuroglancer.LocalVolume(
        data=volume, 
        dimensions=neuroglancer.CoordinateSpace(names=['x', 'y', 'z'], units='nm', scales=[14720, 14720, 20000]), 
        voxel_offset=(0, 0, 0)
    ),
)

with viewer.txn() as s:
#     s.layers.clear()
    s.layers['layer'] = layer

In [ ]:
import tinybrain
from cloudvolume import CloudVolume

volume = volume
factor = (2, 2, 1)
volumes = tinybrain.downsample_segmentation(volume, factor=factor, num_mips=2, sparse=False)
volumes.insert(0, volume)

In [ ]:
path = 'file:///home/sherry/Projects/pipeline_utility/Litao/MD589_shell'
info = CloudVolume.create_new_info(
    num_channels = 1,
    layer_type = 'segmentation',
    data_type = 'uint8', # Channel images might be 'uint8'
    encoding = 'raw', # raw, jpeg, compressed_segmentation, fpzip, kempressed
    resolution = [14720, 14720, 20000], # Voxel scaling, units are in nanometers
    voxel_offset = [0, 0, 0], # x,y,z offset in voxels from the origin
    chunk_size = [512, 512, 16], # units are voxels
    volume_size = volume.shape, # e.g. a cubic millimeter dataset
)
vol = CloudVolume(path, info=info, compress=False, progress=False)

for mip, volume in enumerate(volumes):
    vol.add_scale(np.array(factor) ** mip)
    vol.commit_info()
    vol = CloudVolume(path, mip=mip, compress=False, progress=False)
    vol[:, :, :] = volume[:, :, :]
    
vol.commit_info()